In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
from summaryScraper import SummaryScraper
from powerplayScraper import PowerplayScraper
from shotAttemptsScraper import ShotAttemptsScraper
from selenium.common.exceptions import WebDriverException

In [3]:
scraper = ShotAttemptsScraper()

In [4]:
for i in range(2010, 2022):
    scraper.etl(start=i)
    time.sleep(60)

Loaded 2460 records
Loaded 2460 records
Loaded 1440 records
Loaded 2460 records
Loaded 2460 records
Loaded 2460 records
Loaded 2460 records
Loaded 2542 records
Loaded 2542 records
Loaded 2164 records
Loaded 1736 records
Loaded 2624 records
